### Dataset: Brazilian E-Commerce Public Dataset by Olist

Conjunto de dados públicos de comércio eletrônico brasileiro por Olist

Este é um conjunto de dados públicos de comércio eletrônico brasileiro de pedidos feitos na Olist Store. O conjunto de dados contém informações de 100 mil pedidos de 2016 a 2018 feitos em vários marketplaces no Brasil. Seus recursos permitem visualizar um pedido em múltiplas dimensões: desde status do pedido, preço, desempenho de pagamento e frete até localização do cliente, atributos do produto e finalmente avaliações escritas pelos clientes. Também um conjunto de dados de geolocalização que relaciona os CEPs brasileiros às coordenadas lat/lng.

Estes são dados comerciais reais, foram anonimizados e as referências às empresas e parceiros no texto da revisão foram substituídas pelos nomes das grandes casas de Game of Thrones.

Olist, a maior loja de departamentos dos mercados brasileiros. Olist conecta pequenas empresas de todo o Brasil a canais sem complicações e com um único contrato. Esses comerciantes podem vender seus produtos através da Olist Store e enviá-los diretamente aos clientes usando os parceiros logísticos da Olist. Veja mais em nosso site: www.olist.com

Depois que um cliente compra o produto na Olist Store, um vendedor é notificado para atender ao pedido. Assim que o cliente recebe o produto, ou vence a data estimada de entrega, o cliente recebe por e-mail uma pesquisa de satisfação onde pode dar uma nota da experiência de compra e anotar alguns comentários.
Atenção

Um pedido pode ter vários itens.
Cada item pode ser entregue por um vendedor distinto.
Todos os textos de identificação de lojas e parceiros foram substituídos pelos nomes das grandes casas de Game of Thrones.


PNL:

Este conjunto de dados oferece um ambiente supremo para analisar o texto das avaliações em suas múltiplas dimensões.

Agrupamento:

Alguns clientes não escreveram um comentário. Mas por que eles estão felizes ou bravos?

Previsão de vendas:

Com as informações da data de compra, você poderá prever vendas futuras.

Desempenho de entrega:

Você também poderá trabalhar o desempenho da entrega e encontrar maneiras de otimizar os prazos de entrega.

Qualidade do produto:

Divirta-se descobrindo as categorias de produtos mais propensas à insatisfação do cliente.

Engenharia de recursos:

Crie recursos a partir deste rico conjunto de dados ou anexe algumas informações públicas externas a ele.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.tree import DecisionTreeClassifier

###Analisando as colunas existentes e as relações com as outras tabelas

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_customers_dataset.csv")
df1.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [ ]:
df2=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_geolocation_dataset.csv")
df2.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [ ]:
df2['geolocation_city'].value_counts()

geolocation_city
sao paulo               135800
rio de janeiro           62151
belo horizonte           27805
são paulo                24918
curitiba                 16593
                         ...  
jacuípe                      1
mar vermelho                 1
quebrangulo                  1
poço das trincheiras         1
poxim                        1
Name: count, Length: 8011, dtype: int64

In [ ]:
df2.groupby('geolocation_city').agg({'geolocation_lat': 'mean', 'geolocation_lng': 'mean'}).reset_index()

,geolocation_city,geolocation_lat,geolocation_lng
0,* cidade,-25.571748,-49.334374
1,...arraial do cabo,-22.969370,-42.029834
2,4o. centenario,-24.271860,-53.069433
3,4º centenario,-24.279801,-53.074723
4,abadia de goias,-16.767063,-49.438089
...,...,...,...
8006,áurea,-27.695009,-52.055076
8007,ângulo,-23.195024,-51.917294
8008,érico cardoso,-13.393047,-42.134282
8009,óbidos,-1.907183,-55.516370


In [ ]:
df3=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_order_items_dataset.csv")
df3.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [ ]:
df4=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_order_payments_dataset.csv")
df4.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [ ]:
df5=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_orders_dataset.csv")
df5.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [ ]:
df6=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_products_dataset.csv")
df6.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [ ]:
df7=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_sellers_dataset.csv")
df7.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [ ]:
df8=pd.read_csv("/content/drive/MyDrive/ADA-DS/Machine Learning/Projeto/olist_order_reviews_dataset.csv")
df8.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


####Juntando arquivos

In [ ]:
df=pd.merge(df3,df4,on='order_id')
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,1,credit_card,3,218.04


In [ ]:
df=pd.merge(df,df5,on='order_id')
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,1,credit_card,2,72.19,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,1,credit_card,3,259.83,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,1,credit_card,5,216.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,1,credit_card,2,25.78,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,1,credit_card,3,218.04,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [ ]:
df=pd.merge(df,df6,on='product_id')
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,...,2017-09-20 23:43:48,2017-09-29 00:00:00,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,...,2017-07-13 20:39:29,2017-07-26 00:00:00,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,...,2018-06-04 18:34:26,2018-06-07 00:00:00,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,...,2017-08-09 21:26:33,2017-08-25 00:00:00,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,...,2017-08-24 20:04:21,2017-09-01 00:00:00,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0


In [ ]:
df_copy=pd.merge(df,df7,on='seller_id')
df_copy.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix_x,seller_city_x,seller_state_x,seller_zip_code_prefix_y,seller_city_y,seller_state_y
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP


###Analisando as colunas que permanecerão no dataset e as que serão dropadas.

In [ ]:
df=df_copy.copy()
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,...,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix_x,seller_city_x,seller_state_x,seller_zip_code_prefix_y,seller_city_y,seller_state_y
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
1,130898c0987d1801452a8ed92a670612,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
2,532ed5e14e24ae1f0d735b91524b98b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
3,6f8c31653edb8c83e1a739408b5ff750,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP
4,7d19f4ef4d04461989632411b7e588b9,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,...,650.0,28.0,9.0,14.0,27277,volta redonda,SP,27277,volta redonda,SP


In [ ]:
df.columns

Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value', 'payment_sequential',
       'payment_type', 'payment_installments', 'payment_value', 'customer_id',
       'order_status', 'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix_x',
       'seller_city_x', 'seller_state_x', 'seller_zip_code_prefix_y',
       'seller_city_y', 'seller_state_y'],
      dtype='object')

In [ ]:
df.drop(columns=['order_id', 'order_item_id','seller_id','customer_id','seller_zip_code_prefix_x','seller_city_x','seller_state_x','seller_zip_code_prefix_y'],inplace=True)

In [ ]:
df.head()

,product_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,order_status,order_purchase_timestamp,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_city_y,seller_state_y
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,72.19,delivered,2017-09-13 08:59:02,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
1,4244733e06e7ecb4970a6e2683c13e61,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,73.86,delivered,2017-06-28 11:52:20,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
2,4244733e06e7ecb4970a6e2683c13e61,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,83.23,delivered,2018-05-18 10:25:53,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
3,4244733e06e7ecb4970a6e2683c13e61,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,75.07,delivered,2017-08-01 18:38:42,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
4,4244733e06e7ecb4970a6e2683c13e61,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,72.19,delivered,2017-08-10 21:48:40,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP


###Renomeando colunas

In [ ]:
df.rename(columns={"seller_city_y":"seller_city","seller_state_y":"seller_state"},inplace=True)

###Jutando com a tabela de geolocalização - usar um gráfico por região talvez

In [ ]:
# df=pd.merge(df,df2,on='geolocation_zip_code_prefix')
# df.head()

,product_id,shipping_limit_date,price,freight_value,payment_sequential,payment_type,payment_installments,payment_value,order_status,order_purchase_timestamp,...,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_city,seller_state
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-19 09:45:35,58.9,13.29,1,credit_card,2,72.19,delivered,2017-09-13 08:59:02,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
1,4244733e06e7ecb4970a6e2683c13e61,2017-07-05 02:44:11,55.9,17.96,1,boleto,1,73.86,delivered,2017-06-28 11:52:20,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
2,4244733e06e7ecb4970a6e2683c13e61,2018-05-23 10:56:25,64.9,18.33,1,credit_card,2,83.23,delivered,2018-05-18 10:25:53,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
3,4244733e06e7ecb4970a6e2683c13e61,2017-08-07 18:55:08,58.9,16.17,1,credit_card,3,75.07,delivered,2017-08-01 18:38:42,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
4,4244733e06e7ecb4970a6e2683c13e61,2017-08-16 22:05:11,58.9,13.29,1,credit_card,4,72.19,delivered,2017-08-10 21:48:40,...,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,volta redonda,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117596,f739e9151702508b18f796c53005e5e9,2017-04-27 05:10:28,119.9,16.14,1,boleto,1,136.04,delivered,2017-04-19 10:20:51,...,moveis_decoracao,44.0,533.0,2.0,1000.0,69.0,11.0,11.0,curitiba,PR
117597,f4135cbdece8245560f7be179533797a,2017-08-17 02:55:27,209.0,16.21,1,boleto,1,225.21,delivered,2017-08-11 10:15:22,...,eletronicos,32.0,1306.0,3.0,500.0,17.0,9.0,25.0,curitiba,PR
117598,69c590f7ffc7bf8db97190b6cb6ed62e,2018-08-02 04:05:13,6729.0,193.21,1,boleto,1,6922.21,delivered,2018-07-25 18:10:17,...,pcs,50.0,1935.0,4.0,5660.0,54.0,18.0,47.0,londrina,PR
117599,5ff4076c0f01eeba4f728c9e3fa2653c,2018-04-18 20:10:33,27.9,14.44,1,credit_card,1,84.68,delivered,2018-04-12 19:47:40,...,bebidas,28.0,242.0,1.0,2000.0,19.0,38.0,19.0,sao paulo,SP


In [ ]:
df.columns

Index(['product_id', 'shipping_limit_date', 'price', 'freight_value',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_city', 'seller_state'],
      dtype='object')

####Tentar juntar a geolocalização (latitude e longitude) porém observou-se que existem erros na coluna de cidades, um exemploe é que são paulo está escrito com 2 formas diferentes.

In [ ]:
df['seller_city'].value_counts()

seller_city
sao paulo                29157
ibitinga                  8242
curitiba                  3134
santo andre               3095
sao jose do rio preto     2682
                         ...  
paracambi                    1
resende                      1
sao miguel do oeste          1
ipua                         1
avare                        1
Name: count, Length: 611, dtype: int64